In [1]:
import torch
from Nets import Net, NetNoise
from functions import train, test
from dataloaders import MNIST, MNIST_train_random_lebels, dead_leaves_squares_pretraining
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt

torch.cuda.is_available()

c:\Users\QbaSo\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


True

In [100]:
from torch.utils.data import Dataset
import os
import json
from skimage import io
import torchvision
from PIL import Image

resize_image = True

transform_array = []
if resize_image:
    transform_array.append(
        torchvision.transforms.Resize((96,96))
    )

transform_array += [
    torchvision.transforms.RandomResizedCrop(64, scale=(0.08, 1)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
    torchvision.transforms.RandomGrayscale(p=0.2),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        (0.44087801806139126, 0.42790631331699347, 0.3867879370752931),
        (0.26826768628079806, 0.2610450402318512, 0.26866836876860795),
    ),
]

transform = torchvision.transforms.Compose(transform_array)

class ImageNet100(Dataset):
    def __init__(self, data_root, transform=None):
        train_folders = ["train.X1","train.X2","train.X3","train.X4"]
        f = open("labels.json")
        labels_dict = json.load(f)
        self.data_root = data_root
        self.images = []
        self.labels = []
        self.transform = transform

        for train_folder in train_folders:
            for label in os.listdir(data_root+'\\'+train_folder):
                for img in os.listdir(data_root+'\\'+train_folder+'\\'+label):
                    self.images.append(data_root+'\\'+train_folder+'\\'+label+'\\'+img)
                    self.labels.append(labels_dict[label])
        self.labels = torch.tensor(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
            x = io.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T
            x = torchvision.io.read_image("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")
            x = Image.open("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")
            x = cv2.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

            if self.transform:
                x = self.transform(x)
            return x, self.labels[idx]

In [101]:
# io.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T

In [102]:
# torchvision.io.read_image("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [103]:
# Image.open("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [104]:
# import cv2
# cv2.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG")

In [105]:
# plt.imread("E:\\Datasets\\Imagenet100\\train.X1\\n01440764\\n01440764_10026.JPEG").T

In [106]:
print('-'*50, 'pretraining', '-'*50)
train_loader_random_lebels = dead_leaves_squares_pretraining()

model = NetNoise()
optimizer = optim.SGD(model.parameters(), lr=0.1)
# train_losses = train(model, optimizer, train_loader_random_lebels)
# plt.plot(train_losses)

model.load_state_dict(torch.load("pretrained_dead_leaves_squares.pt"))

-------------------------------------------------- pretraining --------------------------------------------------
Loading data from data/dead_leaves-squares/ as imagefolder


<All keys matched successfully>

In [107]:
print('-'*50, 'training only last layer', '-'*50)
dl = ImageNet100("E:\Datasets\Imagenet100", transform)
train_loader = torch.utils.data.DataLoader(dl, batch_size=1, num_workers=2, shuffle=False)

newModel = model.return_pretrained_model(100)
# train_losses = train(model, optimizer, train_loader)
# test(model, test_loader)

-------------------------------------------------- training only last layer --------------------------------------------------


In [108]:
newModel(dl[0][0])

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [95]:
dl[0][0]

TypeError: Unexpected type <class 'numpy.ndarray'>

In [28]:
type(dl[0][0])

torch.Tensor

In [33]:
train_loader_random_lebels.dataset[0][0].shape

torch.Size([3, 64, 64])

In [19]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [10]:
import torchvision
train_path = "E:\Datasets\Imagenet100\\train.X1"

dataset = torchvision.datasets.ImageFolder(
    train_path,
    transform=None)

In [11]:
dataset

Dataset ImageFolder
    Number of datapoints: 32500
    Root location: E:\Datasets\Imagenet100\train.X1

In [5]:
dl = ImageNet100("E:\Datasets\Imagenet100")
train_loader = torch.utils.data.DataLoader(dl, batch_size=1, num_workers=2, shuffle=False)

In [36]:
# unique_labels = np.unique(dl.labels)

# labels_dict = dict()

# for i in range(len(unique_labels)):
#     labels_dict[unique_labels[i]] = i
    
# f = open("labels.json", "w")

# json.dump(labels_dict, f)
# f.close()


In [68]:
train_loader = torch.utils.data.DataLoader(dl, batch_size=1, num_workers=2, shuffle=False)

In [69]:
train_loader.dataset[0]

(array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [70]:
iter(train_loader)

In [25]:
train_loader_random_lebels = dead_leaves_squares_pretraining()

Loading data from data/dead_leaves-squares/ as imagefolder


In [50]:
train_loader_random_lebels

In [40]:
train_loader_random_lebels.dataset[0]

(tensor([[[-0.7810, -0.7956, -0.5324,  ...,  0.2862,  0.2862,  0.2862],
          [-0.4155, -0.4594, -0.4886,  ...,  0.2862,  0.2862,  0.2862],
          [ 0.0815,  0.0815, -0.1378,  ...,  0.2862,  0.2862,  0.2862],
          ...,
          [ 0.3593,  0.3593,  0.3593,  ...,  0.3593,  0.3593,  0.3593],
          [ 0.3593,  0.3593,  0.3593,  ...,  0.3593,  0.3593,  0.3593],
          [ 0.3593,  0.3593,  0.3593,  ...,  0.3593,  0.3593,  0.3593]],
 
         [[-0.7529, -0.7679, -0.4975,  ...,  0.3438,  0.3438,  0.3438],
          [-0.3773, -0.4224, -0.4524,  ...,  0.3438,  0.3438,  0.3438],
          [ 0.1335,  0.1335, -0.0919,  ...,  0.3438,  0.3438,  0.3438],
          ...,
          [ 0.4189,  0.4189,  0.4189,  ...,  0.4189,  0.4189,  0.4189],
          [ 0.4189,  0.4189,  0.4189,  ...,  0.4189,  0.4189,  0.4189],
          [ 0.4189,  0.4189,  0.4189,  ...,  0.4189,  0.4189,  0.4189]],
 
         [[-0.5785, -0.5931, -0.3303,  ...,  0.4871,  0.4871,  0.4871],
          [-0.2136, -0.2573,

In [2]:
model = NetNoise()

In [3]:
newModel = model.return_pretrained_model(100)

In [7]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [6]:
import torch.nn as nn
newModel.append(nn.Linear())

Sequential(
  (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (2): Dropout2d(p=0.5, inplace=False)
  (3): Linear(in_features=3380, out_features=50, bias=True)
)

In [43]:
import torch.nn as nn
last_layer_in_features = list(list(model.children())[-1].parameters())[0].shape[1]
no_classes = 100

newModel = nn.Sequential(
    *list(model.children())[:-1],
    nn.Linear(last_layer_in_features, no_classes))

In [44]:
newModel

Sequential(
  (0): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (1): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (2): Dropout2d(p=0.5, inplace=False)
  (3): Linear(in_features=3380, out_features=50, bias=True)
  (4): Linear(in_features=50, out_features=100, bias=True)
)

In [25]:
model

NetNoise(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=3380, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=105, bias=True)
)

In [38]:
list(list(model.children())[-1].parameters())[0].shape[1]

50